Stand: 24.02.25

## Darstellung der Messtationsorte auf einer Karte
- 6 verschiedene Orte für den DWD der Station 02712 (blaue Marker)
    - ab dem 19.10.2020 wurde die Station von der Stadt ins Grüne verlegt
    - ID Klimamessstation DWD: 02712 (https://opendata.dwd.de/climate_environment/CDC/observations_germany/) 
- 1 Luftgütemessstation in der Stadt (roter Marker)
- 3 eventuell interessante Messstationen in der Schweiz (grüne Marker)
    - Zugriff auf die Daten eventuell erst ab 01.04.25 möglich, weil noch an der Webseite gearbetet wird

In [1]:
import folium
import pandas as pd
from datetime import datetime
import webbrowser
from pathlib import Path



# Daten laden
# data_dwd = {
#     "Stations_id": [2712, 2712, 2712, 2712, 2712, 2712],
#     "Stationshoehe": [400.00, 398.00, 443.00, 442.50, 442.50, 428.06],
#     "Geogr.Breite": [47.6594, 47.6672, 47.6774, 47.6774, 47.6774, 47.6952],
#     "Geogr.Laenge": [9.1790, 9.1798, 9.1900, 9.1900, 9.1900, 9.1307],
#     "von_datum": ["19510702", "19570801", "19720918", "20070307", "20180927", "20201019"],
#     "bis_datum": ["19570731", "19720917", "20070306", "20180926", "20201018", ""],
#     "Stationsname": ["Konstanz"] * 6
# }

# # Kreuzlingen - Daten seit 2010
# # HAI - Daten seit 1959
# # GUT - Daten seit 1976 - https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&table=false&station=GUT
# data_schweiz = {
#     "Stationshoehe": [410, 720, 442],
#     "Geogr.Breite": [47.647083, 47.651242, 47.601733],
#     "Geogr.Laenge": [9.189364, 9.023911, 9.279428],
#     "von_datum": ["01.09.2010", "01.01.1959", "01.06.1976"],
#     "bis_datum": ["heute", "heute", "heute"],
#     "Stationsname": ["Kreuzlingen", "Salen-Reutenen (HAI)", "Güttingen (GUT)"] 
# }

In [2]:
open_climate_data = pd.read_csv("maja_geodaten/open_climate_data.csv", sep=";")
today = datetime.now().strftime('%Y-%m-%d')
# NaN-Werte in der Spalte "end-datum" mit dem aktuellen Datum ersetzen
open_climate_data['end_datum'] = open_climate_data['end_datum'].fillna(today)

display(open_climate_data.head(6))

dwd_df = open_climate_data[open_climate_data["Quelle"] == "DWD"]
schweiz_df = open_climate_data[open_climate_data["Quelle"] == "MeteoSchweiz"]

,Stations_id,start_datum,end_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,Entfernung_km,Info,Quelle
0,2712.0,1972-09-18,2020-10-18,442.50,47.677400,9.190000,Konstanz,Baden-Württemberg,0.000000,Konstanz vor 2020,DWD
1,2712.0,2020-10-19,2025-03-27,428.06,47.695200,9.130700,Konstanz,Baden-Württemberg,0.000000,Konstanz nach 2020,DWD
2,6258.0,2003-01-01,2025-03-27,461.00,47.684500,9.440900,Friedrichshafen-Unterraderach,Baden-Württemberg,23.248964,Friedrichshafen-Unterraderach,DWD
3,6263.0,2004-07-01,2025-03-27,445.00,47.773800,8.821900,Singen,Baden-Württemberg,24.692402,Singen,DWD
4,NaN,2010-09-01,2025-03-27,410.00,47.647083,9.189364,Kreuzlingen,Schweiz,NaN,Kreuzlingen,MeteoSchweiz
5,NaN,1959-01-01,2025-03-27,720.00,47.651242,9.023911,Salen-Reutenen (HAI),Schweiz,NaN,Salen-Reutenen,MeteoSchweiz


In [3]:
import folium
import pandas as pd
from datetime import datetime
import webbrowser
from pathlib import Path

# --- Daten einlesen & vorbereiten ---
open_climate_data = pd.read_csv("maja_geodaten/open_climate_data.csv", sep=";")
today = datetime.now().strftime('%Y-%m-%d')
open_climate_data['end_datum'] = open_climate_data['end_datum'].fillna(today)

# Aufteilen nach Quelle
dwd_df = open_climate_data[open_climate_data["Quelle"] == "DWD"]
schweiz_df = open_climate_data[open_climate_data["Quelle"] == "MeteoSchweiz"]


# --- Hilfsfunktion: Datum formatieren ---
def format_date(date_str):
    try:
        return pd.to_datetime(date_str).strftime("%d.%m.%Y")
    except:
        return str(date_str)

# --- Neue Funktion: Stationen als Marker zur Karte hinzufügen ---
def add_station_markers(map_obj, df, source_name, marker_color="blue", text_color="#000000", show_info=True):
    """
    Fügt Stationen aus einem DataFrame zur Karte hinzu.

    map_obj       : folium.Map – die Zielkarte
    df            : pd.DataFrame – enthält geoBreite, geoLaenge, start_datum, end_datum, Stationshoehe, Info
    source_name   : str – z. B. 'DWD' oder 'Schweiz'
    marker_color  : str – Farbe des Icons
    text_color    : str – Hex-Farbe für den Info-Text
    show_info     : bool – 'Info'-Text als DivIcon anzeigen?
    """
    grouped = df.sort_values(by='start_datum').groupby(['geoBreite', 'geoLaenge'])

    for (lat, lon), group in grouped:
        popup_text = "<br><br>".join([
            f"<b>{row.get('Stationsname', row.get('Info', ''))}</b><br>"
            f"Von: {format_date(row['start_datum'])}<br>"
            f"Bis: {format_date(row['end_datum'])}<br>"
            f"Höhe: {row['Stationshoehe']} m"
            for _, row in group.iterrows()
        ])

        popup = folium.Popup(popup_text, max_width=300)
        tooltip = f"{source_name}: {group.iloc[0].get('Stationsname', group.iloc[0].get('Info', ''))}"

        # Hauptmarker
        folium.Marker(
            location=[lat, lon],
            tooltip=tooltip,
            popup=popup,
            icon=folium.Icon(color=marker_color)
        ).add_to(map_obj)

        # Info-Text als HTML-Overlay
        if show_info:
            info_list = [row.get("Info", "") for _, row in group.iterrows() if pd.notna(row.get("Info", ""))]

            if info_list:
                info_html = '<div style="white-space: nowrap; text-align: center;">' + ''.join([
                    f'''
                    <span style="
                        display: inline-block;
                        margin-right: 10px;
                        font-size: 12pt;
                        font-weight: bold;
                        color: {text_color};
                        text-shadow: -1px -1px 0 #000, 1px -1px 0 #000,
                                     -1px 1px 0 #000, 1px 1px 0 #000;">
                        {info}
                    </span>
                    '''
                    for info in info_list
                ]) + '</div>'

                folium.Marker(
                    [lat + 0.01, lon + 0.016],
                    icon=folium.DivIcon(html=info_html)
                ).add_to(map_obj)



# --- Karte erstellen ---
m = folium.Map(location=[dwd_df["geoBreite"].mean(), dwd_df["geoLaenge"].mean()], zoom_start=11)



# --- Luftgüte-Station (rot) ---
luftguete_lat = 47 + 39/60 + 51.70/3600
luftguete_lon = 9 + 10/60 + 9.44/3600

# folium.Marker(
#     location=[luftguete_lat, luftguete_lon],
#     tooltip="Luftgüte",
#     popup="Luftgüte",
#     icon=folium.Icon(color='red')
# ).add_to(m)


# Konstanz Karte einfügen
# Optional: Stil-Funktion für farbige Darstellung
def style_function(feature):
    return {
        'fillOpacity': 0.1,
        'weight': 1.5,
        'color': '#483D8B', # darkgrey #3b3b3b
        'fillColor': '#808080'
    }

# Optional: Popup mit Stadtteilnamen
def popup_function(feature):
    name = feature['properties'].get('Name', 'Stadtteil')
    return folium.Popup(name)

geojson_path = "maja_geodaten/Kleinräumige_Gliederung_4330766187849128049.geojson"
# GeoJSON-Objekt hinzufügen
folium.GeoJson(
    geojson_path,
    name="Stadtteile Konstanz",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["STT_NAME"], aliases=["Stadtteil:"]),
    popup=popup_function
).add_to(m)

# --- Marker einfügen ---
add_station_markers(
    m,
    schweiz_df,
    source_name="Schweiz",
    marker_color="green",
    text_color="#6EA925",  # grünlich
    show_info=True
)

add_station_markers(
    m,
    dwd_df,
    source_name="DWD",
    marker_color="blue",
    text_color="#37A7D9",  # hellblau
    show_info=True
)

# --- Karte speichern & im Browser öffnen ---
data_dir = Path.cwd() / "../data"
data_dir.mkdir(parents=True, exist_ok=True)
map_file = data_dir / "konstanz_map.html"
m.save(map_file)
webbrowser.open(map_file.as_uri())


True